In [125]:
import numpy as np
import random as rand 
from numpy.linalg import norm
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq
import struct
import pyaudio
import time
from numpy.linalg import norm
import seaborn as sns

%matplotlib tk

In [128]:
class AudioCapture:
    
    # max settings 
    CHUNK = 1024 * 10
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 192000

    def __init__(self):
        
        # audio
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(
            format=self.FORMAT,
            channels=self.CHANNELS,
            rate=self.RATE,
            input=True,
            output=True,
            frames_per_buffer=self.CHUNK)
        
        
        self.x_freq = np.linspace(0, self.RATE, self.CHUNK)
        self.FFT = np.zeros(len(self.x_freq))
        self.t0 = time.time()
        self.intervals = [(20,300), (300,1500)] 
        #self.intervals = [(20,600), (21,600)] 
        #self.intervals = [(20,1500), (21,1500)]
        self.bead_count = [3000, 3000]
        self.sens = [0.05, 0.05]
        self.max_size = [2,7]
        self.max_freqs = [0 for i in range(len(self.intervals))]
        self.decay = [0.02, 0.006]
        self.mesh = self.make_mesh()
        self.beads = self.make_beads()
        self.clrs = sns.color_palette(sns.diverging_palette(10, 220, sep=80, n=self.CHUNK))  # a list of RGB tuples

        self.leads = {intrv: {'x': rand.uniform(0,1), 'y': rand.uniform(0,1), 
                                 'vx': 0, 'vy': 0, 
                                 'mass': .001} for intrv in self.intervals}
        
       
    def play(self):
        plt.style.use('dark_background')
        fig, (ax0, ax2) = plt.subplots(2, figsize = (10, 12))
        
        # fft plot and smoothed
        fourier, = ax0.semilogx(self.x_freq, np.zeros(self.CHUNK), '-', c = 'c', lw = 2)
        ax0.set_ylim(-.1, 1)
        ax0.set_xlim(20, self.RATE/2)
        ax0.set_xlabel( 'frequency' )
                                  
        ''' I guess I just have to make a line for each damn group I create'''
        # visualization 
        # https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.plot.html#matplotlib.axes.Axes.plot
        # https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.lines.Line2D.html
        beads1, = ax2.plot([], [], 'ro', markersize = 1, marker = '.') 
        beads2, = ax2.plot([], [], 'co', markersize = 1, marker = '.') 
        spwner1, = ax2.plot([], [], 'ro', markersize = 1, marker = '.')
        spwner2, = ax2.plot([], [], 'co', markersize = 1, marker = '.')
        
        ax2.set_xlim(0,1)
        ax2.set_ylim(0,1)
        ax2.set_xticks([])
        ax2.set_yticks([])
        
        while True:
            
            # audio
            C = 300000
            data = self.stream.read(self.CHUNK)
            data_int = np.array(struct.unpack(str(2 * self.CHUNK) + 'B', data), dtype='b')[::2]
            self.FFT = abs(fft(data_int)/C)
            fourier.set_ydata(self.FFT[:self.CHUNK])
                                    
            # ANIMATION
            marker_sizes = self.update_points(self.FFT)
            self.border_adjustment()
            
            # spawners
            X, Y = self.x_y_unpack(self.leads)
            spwner1.set_data(X[0],Y[0])
            
            spwner2.set_data(X[1],Y[1])
            
            # beads
            X, Y = self.x_y_unpack(self.beads)
            beads1.set_data(X[0],Y[0])
            beads1.set_markersize(marker_sizes[0])
            beads2.set_data(X[1],Y[1])
            beads2.set_markersize(marker_sizes[1])
         
            # plot
            fig.canvas.draw()
            fig.canvas.flush_events()  
            
    # change from dicitonary that need to be unpacked to a straight up np array that can be += in on line
    def update_points(self, FFT):
        
        threshold_beads = .01
        threshold_spawner = .1
        xx = 0 ; yy = 0
        
        dt = (time.time()-self.t0)/30
        
        for i in range(len(self.intervals)):
            intrvl = self.intervals[i] 
            
            # medium
            d_x = self.beads[intrvl]['x']-self.leads[intrvl]['x']
            d_y = self.beads[intrvl]['y']-self.leads[intrvl]['y']
            dist = np.sqrt(d_x**2 + d_y**2)/10
            self.beads[intrvl]['x'] += (1/dist)*np.cos(dt)*.001
            self.beads[intrvl]['y'] += (1/dist)*np.sin(dt)*.001
            
            # angle action
            mesh_i = self.mesh[i]
            for j in range(len(mesh_i)):
                mesh_ij = mesh_i[j]
                index = mesh_ij[1][0]
                
                # point color and smoothener
                self.max_freqs[i] -= self.decay[i]
                if FFT[index] >= self.max_freqs[i]:
                    self.max_freqs[i] = FFT[index]
                    
                if (FFT[index] >= threshold_beads):
                    theta = mesh_ij[0]
                    xx += np.cos(theta) * FFT[index]*self.sens[i] 
                    yy += np.sin(theta) * FFT[index]*self.sens[i] 
                    self.leads[intrvl]['x'] += xx*.001
                    self.leads[intrvl]['y'] += yy*.001
            
        max_freqs = [1+self.max_size[i]*self.max_freqs[i] for i in range(len(self.intervals))]
        return max_freqs
            
                                             
    def x_y_unpack(self, data):
        X, Y = [], []
        for key in data.keys():
            group = data[key]
            X.append(group['x'])
            Y.append(group['y'])
        return X, Y
    
    def border_adjustment(self):
        delta = 0.0001
        
        for intrvl in self.intervals:
            # leads
            xl = self.leads[intrvl]['x']
            yl = self.leads[intrvl]['y'] 
            if 1-xl < delta:
                self.leads[intrvl]['x'] -= 1
            if 1-yl < delta:
                self.leads[intrvl]['y'] -= 1
            if xl < delta:
                self.leads[intrvl]['x'] += 1
            if yl < delta:
                self.leads[intrvl]['y'] += 1
              
            # beads
            for i in range(len(self.beads[intrvl]['x'])):
                xb = self.beads[intrvl]['x'][i] 
                yb = self.beads[intrvl]['y'][i] 
                if 1-xb < delta:
                    self.beads[intrvl]['x'][i] -= 1
                if 1-yb < delta:
                    self.beads[intrvl]['y'][i] -= 1
                if xb < delta:
                    self.beads[intrvl]['x'][i] += 1
                if yb < delta:
                    self.beads[intrvl]['y'][i] += 1
                    
    def make_mesh(self):
        # indexing frequencies for ease of subsetting and processing
        freq = np.linspace(0, self.RATE, self.CHUNK)
        freq = [(i, freq[i]) for i in range(len(freq))]
        
        # mapping n intervals of the frequency to [0,2pi]
        mesh = []
        for intrvl in self.intervals:
            f_scaled = [i for i in freq if (i[1] > intrvl[0] and i[1] < intrvl[1])]
            angle = np.linspace(0, 2*np.pi, len(f_scaled))
            mesh_i = list(zip(angle, f_scaled))
            mesh.append(mesh_i)
        return mesh
    
    def make_beads(self):
        beads = {}
        for i in range(len(self.intervals)):
            count = self.bead_count[i]
            beads.update(
                {self.intervals[i]: {'x': np.array([rand.uniform(0,1) for i in range(count)]),
                          'y': np.array([rand.uniform(0,1) for i in range(count)]),
                          'vx': np.array([rand.uniform(-0.002,0.002) for i in range(count)]),
                          'vy': np.array([rand.uniform(-0.002,0.002) for i in range(count)])}})
        return beads 
         
    def close(self):
        self.stream.close()
        self.p.terminate()
    

In [129]:
audio = AudioCapture()

audio.play()
audio.close()

TclError: invalid command name "pyimage613"